#### Here, we are going to investigate the cached_escape function in cached_semantics.py to reproduce the AUC result

In [1]:
# 0. look up auc function in sklearn
from sklearn.metrics import auc
import numpy as np
# Example data (FPR, TPR)
fpr = np.array([0.0, 0.1, 0.2, 0.3, 0.4, 1.0])
tpr = np.array([0.0, 0.4, 0.6, 0.7, 0.8, 1.0])

# Calculate AUC
roc_auc = auc(fpr, tpr)
print(f"ROC AUC: {roc_auc}")
# I finded out this function simply gets the area under the curve of the ROC curve described by the fpr and tpr arrays

ROC AUC: 0.75


In [15]:
import pandas as pd
import scipy.stats as ss
cov_cscs_path = "../results/cov/semantics/analyze_semantics_cov_bilstm_512.txt"
rbd_cscs_path = "../results/cov/semantics/analyze_semantics_cov2rbd_bilstm_512.txt"
cov_df = pd.read_csv(cov_cscs_path, sep="\t")
rbd_df = pd.read_csv(rbd_cscs_path, sep="\t")
print(cov_df)
len(cov_df)

       pos wt mut          prob  change  is_viable  is_escape
0        0  M   A  2.190679e-06  1389.0      False      False
1        0  M   B  1.134475e-18  1628.0      False      False
2        0  M   C  1.529340e-06  3214.0      False      False
3        0  M   D  6.757930e-09  1710.0      False      False
4        0  M   E  1.990447e-07  1568.0      False      False
...    ... ..  ..           ...     ...        ...        ...
30547    0  M   V  8.076018e-06   765.0      False      False
30548    0  M   W  1.012355e-09  1172.0      False      False
30549    0  M   X  1.913051e-11   436.5      False      False
30550    0  M   Y  2.763461e-07   586.5      False      False
30551    0  M   Z  7.534515e-13  1504.0      False      False

[30552 rows x 7 columns]


30552

In [18]:
# 1. before start test the main function in cached_semantics.py first
def get_aucs(df: pd.DataFrame):
    """What we need is sars-cov-2 related results, so we ignore codes not related 
    to sars-cov-2.
    wt_seq & seqs_escape are used to calculate AUC.
    """
    """get prob & orig_prob from table
    escape_rank_dist, escape_idx
    acquisition = CSCS score (rank base)
     = apply rank to the array(grammar and semantic change) and sum

    target values: norm_auc, norm_auc_prob, norm_auc_change

    for RBD, ignore pos < 330 or pos > 530
    """
    grammaticalities = df['prob']
    semantic_changes = df['change']

    gram_ranks = grammaticalities.rank(ascending=False)
    sem_ranks = semantic_changes.rank(ascending=False)

    escape_indices = df[df['is_escape'] == True].index
    # acquisition = cscs_scores
    cscs_scores = gram_ranks + sem_ranks
    len_probs = len(grammaticalities)
    num_to_consider = list(range(1, len(grammaticalities) + 1))
    
    # escape mutants rank of cscs score
    escape_ranks_cscs = cscs_scores.rank()[escape_indices]
    escape_ranks_gram = gram_ranks[escape_indices]
    escape_ranks_sem = sem_ranks[escape_indices]
    n_escape_cscs = [sum(escape_ranks_cscs <= i + 1) for i in range(len_probs)]
    n_escape_gram = [sum(escape_ranks_gram <= i + 1) for i in range(len_probs)]
    n_escape_sem = [sum(escape_ranks_sem <= i + 1) for i in range(len_probs)]

    norm = (len_probs + 1) * len(escape_ranks_cscs)

    # auc of cscs
    norm_auc_cscs = auc(num_to_consider, n_escape_cscs) / norm
    # auc of grammaticalities 
    norm_auc_gram = auc(num_to_consider, n_escape_gram) / norm
    # auc of semantic change 
    norm_auc_sem = auc(num_to_consider, n_escape_sem) / norm
    
    return (norm_auc_cscs, norm_auc_gram, norm_auc_sem)
result = get_aucs(cov_df)
result

(0.8659576887100414, 0.8390053866706172, 0.6598051358553816)

In [13]:
"""finding out acq_argsort & escape_rank_dist
escape_idx is array of index.
"""
# acq_argsort = ss.rankdata(-acquisition)
# escape_rank_dist = acq_argsort[escape_idx]


TypeError: unsupported operand type(s) for +: 'range' and 'int'